In [7]:
from pysolar.solar import get_altitude, get_azimuth
import datetime
import pytz
import string
import random
import json
import math
import ephem

def get_moon_Elevation_Azimuth(Year,Month,Date,Hour,Minute,Sec,lat,lng):
    # Define your location (latitude and longitude)
    latitude = lat  # Example: San Francisco, CA
    longitude = lng

    # Define the custom date and time
    custom_date = datetime.datetime(Year,Month, Date, Hour, Minute, Sec) 

    # Specify the timezone using pytz (replace with your desired timezone)
    timezone = pytz.timezone(pytz.country_timezones['IN'][0])

    # Make the custom date and time timezone-aware
    localized_date = timezone.localize(custom_date)

    # Set up observer's location
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)
    observer.date = localized_date

    # Create a Moon object
    moon = ephem.Moon()

    # Compute the moon's position for the observer's location and time
    moon.compute(observer)

    # Get the moon's altitude and azimuth in degrees
    moon_altitude = moon.alt * 180 / ephem.pi  # Convert from radians to degrees
    moon_azimuth = moon.az * 180 / ephem.pi # Convert from radians to degrees
    mode='Night'
    print(f"Moon Altitude: {moon_altitude:.2f} degrees")
    print(f"Moon Azimuth: {moon_azimuth:.2f} degrees")

    return moon_altitude,moon_azimuth,mode
    #print(f"Moon Altitude: {moon_altitude:.2f} degrees")
    #print(f"Moon Azimuth: {moon_azimuth:.2f} degrees")


def get_Elevation_Azimuth(Year,Month,Date,Hour,Minute,Sec,lat,lng,country_code='IN',mode='Day'):
    # Example coordinates
    if mode !='Day':
        return get_moon_Elevation_Azimuth(Year,Month,Date,Hour,Minute,Sec,lat,lng)
    else:
        
        latitude = lat 
        longitude = lng

        # Current time in UTC (make timezone-aware using pytz)
        timezone = pytz.timezone(pytz.country_timezones['IN'][0])  
        local_time = datetime.datetime.now() 
        #print(local_time) # Get current local time
        custom_date = datetime.datetime(Year,Month, Date, Hour, Minute, Sec) 
        localized_time = timezone.localize(custom_date)  # Make it timezone-aware

        # Calculate solar position
        elevation = get_altitude(latitude, longitude, localized_time)
        azimuth = get_azimuth(latitude, longitude, localized_time)
        if elevation >0:
            mode='Day'
            print(f"Sun Altitude: {elevation:.2f} degrees")
            print(f"Sun Azimuth: {azimuth:.2f} degrees")
            return azimuth , elevation,mode
        else:
            return get_moon_Elevation_Azimuth(Year,Month,Date,Hour,Minute,Sec,lat,lng)
            
       

        #print(f"Solar Elevation: {elevation:.2f} degrees")
        #print(f"Solar Azimuth: {azimuth:.2f} degrees")
        
       

def xor_shift_encrypt(message, key, shift_value):
    encrypted = ''.join([chr((ord(char) ^ key) << shift_value) for char in message])
    return encrypted
def xor_shift_decrypt(encrypted_message, key, shift_value):
    decrypted = ''.join([chr((ord(char) >> shift_value) ^ key) for char in encrypted_message])
    return decrypted

def get_date_time_latlng():
    date=str(datetime.datetime.now())
    date_time=date.split()
    date=date_time[0].split("-")
    time=date_time[1].split(":")
    #lat lng
    lat=random.uniform(-90,90)
    lng=random.uniform(-180,180)
    #print(date_time)
    date_time={"date":date,"time":time,"latitude":lat,"longitude":lng}
    return date_time


# Key encryption
def xor_key_encrypt(message, key):
    ascii_pwd=[]
    upper_case_letter=list(string.ascii_uppercase)
    lower_case_letter=list(string.ascii_lowercase)
    for i in message:
        ascii_words=[]
        for j in i:
            #converting words to ascii state
            #print(ord(j))
            ascii_words.append(int(ord(j)*int(key)))
            #print("\n")
        ascii_pwd.append(ascii_words)
        #print(ascii_pwd)

    ascii_word=[]
    for i in ascii_pwd:
        
        for j in i:
            while int(j)>0:
                #print(j)
                digit=int(j%10)
                ascii_word.append(upper_case_letter[digit])
                j=j//10
            ascii_word.append('d%')
        ascii_word.append('AbC%')
        #print(ascii_word)

    #2nd layer encryption
    encryption_form="".join(ascii_word)
    encryption_form=encryption_form+chr(len(message))
    #print(encryption_form)
    return encryption_form


# key decryption
def xor_key_decrypt(encrypted_message, key):
    upper_case_letter=list(string.ascii_uppercase)
    lower_case_letter=list(string.ascii_lowercase)
    text=encrypted_message
    text=text.split('AbC%') 
    text.pop()
    i=0
    correct_word_order=[]
    while i<len(text):
        word=text[i].split('d%')
        word.pop()
       # print(word)
        correct_letter_order=[]
        for o in word:
            temp_text=[]
            temp1_text=""
            for j in reversed(range(len(o))):
                
                temp_text.append(o[j])
                temp1_text="".join(temp_text)
            correct_letter_order.append(temp1_text)
    
  
        i+=1 
        correct_word_order.append(correct_letter_order)
    
    ascii_num1=[]
    for a in correct_word_order:
        ascii_num=[]
        for d in a:
            #print(d)
            temp_num=[]
            temp1_num=""
            for k in d:
                #print(k)
                if k in upper_case_letter:
                   #print(lower_case_letter.index(k))
                   temp_num.append(upper_case_letter.index(k))
                   temp1_num="".join(map(str,temp_num))
                   #print(temp1_num)
            ascii_num.append(int(int(temp1_num)//int (key)))
        ascii_num1.append(ascii_num)
    final_list=[]
    for v in ascii_num1:
        temp_word=[]
        final_text=""
       
        for y in v:
            character=chr(y)
            temp_word.append(character)
            final_text="".join(temp_word)
        final_list.append(final_text)
    #print(final_list)

    text="".join(final_list)
    return text

def encrpt_key(date_time_latlng):
    date_time_text=json.dumps(date_time_latlng)
    #print(date_time_text)
    #date_time_dict=json.loads(date_time_text)
    secret_key=xor_key_encrypt(date_time_text,len(date_time_text))
    return secret_key
def decrypt_key(secretkey):
    #key=encrpt_key(secretkey)
    key=secretkey
    value=xor_key_decrypt(key,ord(key[-1]))
    value=json.loads(value)
    
    return value
    
try:
    # encyption text
    def encrytion_text(message):
        pwd=message
        pwd=pwd.split()
        ascii_pwd=[]
        encrytion_form=""
        date_time_latlng=get_date_time_latlng() # returns  Dictionary
        date=date_time_latlng["date"] #Current Date
        time=date_time_latlng["time"]# Current TIme 
        lat=date_time_latlng["latitude"] # Random latitude
        lng=date_time_latlng["longitude"] # Random logitude
        
        
        lower_case_letter=list(string.ascii_lowercase)
        #Getting Sun's position
        secret_value_key,secret_shift_value,mode =get_Elevation_Azimuth(int(date[0]),int(date[1]),int(date[2]),int(time[0]),int(time[1]),math.ceil(float(time[2])),lat,lng)
        date_time_latlng['mode']=mode
        #!----1st layer encryption ----!
        #converting sentence into ascii state...
        for i in pwd:
            ascii_words=[]
            for j in i:
                #converting words to ascii state
                #print(ord(j))
                ascii_words.append(int(ord(j)*int(secret_value_key)))
                #print("\n")
            ascii_pwd.append(ascii_words)
            #print(ascii_pwd)
        ascii_word=[]
        for i in ascii_pwd:
            
            for j in i:
                while int(j)>0:
                    #print(j)
                    digit=int(j%10)
                    ascii_word.append(lower_case_letter[digit])
                    j=j//10
                ascii_word.append('D%')
            ascii_word.append('aBc%')
        encrytion_form="".join(ascii_word)    
        #!-----End of 1st layer encryption -----!
        #!-----2nd layer encrytion ------!
        encrypted_message=xor_shift_encrypt(encrytion_form,int(secret_value_key),1)
        #!-----End of 2nd layer encrption ------!
        Secret_key=encrpt_key(date_time_latlng)
        print(date)
        print(time)
        print(lat)
        print(lng)
        
        return encrypted_message , Secret_key       
except ValueError as e:
    print("Something went wrong!")    
  

try:
    def decryption_text(message,Skey):
        text=message
        date_time_latlng=decrypt_key(Skey)
        print("------")
        print(date_time_latlng)
        date=date_time_latlng["date"] #Current Date
        time=date_time_latlng["time"]# Current TIme 
        lat=date_time_latlng["latitude"] # Random latitude
        lng=date_time_latlng["longitude"] # Random logitude
        mode=date_time_latlng["mode"]#mode
    
        
        secret_value,secret_shift_value,mode=get_Elevation_Azimuth(int(date[0]),int(date[1]),int(date[2]),int(time[0]),int(time[1]),math.ceil(float(time[2])),lat,lng,mode=mode)
        lower_case_letter=list(string.ascii_lowercase)
        ascii_word=[]
        #print(int(secret_value))
        #print(text)
        text=xor_shift_decrypt(text,int(secret_value),1)
        text=text.split('aBc%') 
        text.pop()
        i=0
        correct_word_order=[]
        while i<len(text):
            word=text[i].split('D%')
            word.pop()
        # print(word)
            correct_letter_order=[]
            for o in word:
                temp_text=[]
                temp1_text=""
                for j in reversed(range(len(o))):
                    
                    temp_text.append(o[j])
                    temp1_text="".join(temp_text)
                correct_letter_order.append(temp1_text)
            i+=1 
            correct_word_order.append(correct_letter_order)
        
        ascii_num1=[]
        for a in correct_word_order:
            ascii_num=[]
            for d in a:
                #print(d)
                temp_num=[]
                temp1_num=""
                for k in d:
                    #print(k)
                    if k in lower_case_letter:
                        #print(lower_case_letter.index(k))
                        temp_num.append(lower_case_letter.index(k))
                        temp1_num="".join(map(str,temp_num))
                ascii_num.append(int(int(temp1_num)/int (secret_value)))
            ascii_num1.append(ascii_num)
        final_list=[]
        for v in ascii_num1:
            temp_word=[]
            final_text=""
        
            for y in v:
                character=chr(y)
                temp_word.append(character)
                final_text="".join(temp_word)
            final_list.append(final_text)
        print(final_list)

        text=" ".join(final_list)
        return text     
except Exception as e:
    print("Something went worng!")


text, key =encrytion_text("I am a boy")
print(text)
print(key)
actualtext= decryption_text(text,key)
print(actualtext)




#print(key)
    
            
    
            
            
            
            
    
    
                   
    
    
    
    
            
    
                     
            



    
    


Sun Altitude: 23.12 degrees
Sun Azimuth: 260.32 degrees
['2024', '10', '18']
['19', '45', '00.128169']
-2.612584609984765
29.51201006040577
ˊ˚˜˚ˌʀɂˊʌˎɂˊˎˎ˄ˎʀɂˊ˂ˀ˚ˎʀɂˊʌˎɂˊˎˎ˄ˎʀɂˊʌˎɂˊ˚˂˄ˎʀɂˊˆ˚˚ˎʀɂˊˆ˂ˌˀʀɂˊʌˎɂ
CBHHBd%AbC%GJIEd%AbC%AAEEBd%AbC%IGJDBd%AbC%EAHGBd%AbC%EEFEBd%AbC%GJIEd%AbC%CFDId%AbC%IAGEd%AbC%EABDBd%AbC%GJIEd%AbC%AACHd%AbC%CBJGd%AbC%AACHd%AbC%IIEHd%AbC%GJIEd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%GFAHd%AbC%CBJGd%AbC%GJIEd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%GFAHd%AbC%EGAId%AbC%GJIEd%AbC%CJDDBd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%EAHGBd%AbC%ACBFBd%AbC%GJGFBd%AbC%EEFEBd%AbC%GJIEd%AbC%CFDId%AbC%IAGEd%AbC%EABDBd%AbC%GJIEd%AbC%GFAHd%AbC%IACId%AbC%GJIEd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%IIEHd%AbC%CDGHd%AbC%GJIEd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%CBJGd%AbC%CBJGd%AbC%ECGGd%AbC%GFAHd%AbC%AACHd%AbC%EGAId%AbC%GFAHd%AbC%GHHHd%AbC%IACId%AbC%GJIEd%AbC%CJDDBd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%CFFFBd%AbC%IGJDBd%AbC%EAHGBd%AbC%ACBFBd%AbC%EAHGBd%AbC%IEIGBd%AbC%AAEEBd%AbC%EEFEBd%AbC%GJIEd%AbC%CFDId%AbC%IAG

c:\Users\tusha\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysolar\solartime.py:113: UserWarning: I don't know about leap seconds after 2023
  warnings.warn \


In [6]:
import ephem
import pytz
import datetime


def get_moon_Elevation_Azimuth(Year,Month,Date,Hour,Minute,Sec,lat,lng):
    # Define your location (latitude and longitude)
    latitude = lat  # Example: San Francisco, CA
    longitude = lng

    # Define the custom date and time
    custom_date = datetime.datetime(Year,Month, Date, Hour, Minute, Sec) 

    # Specify the timezone using pytz (replace with your desired timezone)
    timezone = pytz.timezone(pytz.country_timezones['IN'][0])

    # Make the custom date and time timezone-aware
    localized_date = timezone.localize(custom_date)

    # Set up observer's location
    observer = ephem.Observer()
    observer.lat = str(latitude)
    observer.lon = str(longitude)
    observer.date = localized_date

    # Create a Moon object
    moon = ephem.Moon()

    # Compute the moon's position for the observer's location and time
    moon.compute(observer)

    # Get the moon's altitude and azimuth in degrees
    moon_altitude = moon.alt * 180 / ephem.pi  # Convert from radians to degrees
    moon_azimuth = moon.az * 180 / ephem.pi    # Convert from radians to degrees

    print(f"Moon Altitude: {moon_altitude:.2f} degrees")
    print(f"Moon Azimuth: {moon_azimuth:.2f} degrees")

get_moon_Elevation_Azimuth(2024, 10, 18, 18, 30, 0,20.593683 ,78.962883)
azimuth, Elevation=get_Elevation_Azimuth(2024, 10, 18, 18, 30, 0,20.593683 ,78.962883)
print(f"Sun Altitude: {Elevation:.2f} degrees")
print(f"Sun Azimuth: {azimuth:.2f} degrees")



Moon Altitude: 0.43 degrees
Moon Azimuth: 71.27 degrees


NameError: name 'get_Elevation_Azimuth' is not defined

In [8]:
import base64

# Original string
original_string = key

# Encode the string (transform it)
encoded_string = base64.b64encode(original_string.encode())

print(f"Encoded string: {encoded_string}")

# Decode the string (revert to original)
decoded_string = base64.b64decode(encoded_string).decode()

print(f"Decoded string: {decoded_string}")
print(f"Is decoded string the same as original? {decoded_string == original_string}")

Encoded string: b'Q0JISEJkJUFiQyVHSklFZCVBYkMlQUFFRUJkJUFiQyVJR0pEQmQlQWJDJUVBSEdCZCVBYkMlRUVGRUJkJUFiQyVHSklFZCVBYkMlQ0ZESWQlQWJDJUlBR0VkJUFiQyVFQUJEQmQlQWJDJUdKSUVkJUFiQyVBQUNIZCVBYkMlQ0JKR2QlQWJDJUFBQ0hkJUFiQyVJSUVIZCVBYkMlR0pJRWQlQWJDJUdEREdkJUFiQyVJQUdFZCVBYkMlR0pJRWQlQWJDJUdGQUhkJUFiQyVDQkpHZCVBYkMlR0pJRWQlQWJDJUdEREdkJUFiQyVJQUdFZCVBYkMlR0pJRWQlQWJDJUdGQUhkJUFiQyVFR0FJZCVBYkMlR0pJRWQlQWJDJUNKRERCZCVBYkMlR0RER2QlQWJDJUlBR0VkJUFiQyVHSklFZCVBYkMlRUFIR0JkJUFiQyVBQ0JGQmQlQWJDJUdKR0ZCZCVBYkMlRUVGRUJkJUFiQyVHSklFZCVBYkMlQ0ZESWQlQWJDJUlBR0VkJUFiQyVFQUJEQmQlQWJDJUdKSUVkJUFiQyVHRkFIZCVBYkMlSUFDSWQlQWJDJUdKSUVkJUFiQyVHRERHZCVBYkMlSUFHRWQlQWJDJUdKSUVkJUFiQyVJSUVIZCVBYkMlQ0RHSGQlQWJDJUdKSUVkJUFiQyVHRERHZCVBYkMlSUFHRWQlQWJDJUdKSUVkJUFiQyVDQkpHZCVBYkMlQ0JKR2QlQWJDJUVDR0dkJUFiQyVHRkFIZCVBYkMlQUFDSGQlQWJDJUVHQUlkJUFiQyVHRkFIZCVBYkMlR0hISGQlQWJDJUlBQ0lkJUFiQyVHSklFZCVBYkMlQ0pEREJkJUFiQyVHRERHZCVBYkMlSUFHRWQlQWJDJUdKSUVkJUFiQyVDRkZGQmQlQWJDJUlHSkRCZCVBYkMlRUFIR0JkJUFiQyVBQ0JGQmQlQWJDJUVBSEdCZCVBYk

In [10]:
import zlib

# Original string
original_string = key

# Compress the string
compressed_data = zlib.compress(original_string.encode())

print(f"Compressed data: {compressed_data}")
print(f"Compressed size: {len(compressed_data)} bytes")

# Decompress the string
decompressed_data = zlib.decompress(compressed_data).decode()

print(f"Decompressed string: {decompressed_data}")
print(f"Is decompressed string the same as original? {decompressed_data == original_string}")


Compressed data: b'x\x9c\xb5\x94Q\x0e\x03!\x08DO\xd3\x83 "\xe01\x9a\x1e\xa8g\xea\xc9\xfaQu\xc8j\xccn\xd3\xfeMp\x87\x85\'\xc8\xc9,=nt\xe7\x9bV\x97\x8f"\x12iA\xd7\x9a\x9b\x142\xedR\x8aL..\xd9\x9b\x89T\xba\'\xe5Ez\xb6fIU\x8f1w\xb1\xa3Cs\xd6c\xeapZh\xcaw\xdd+J>\xb5T\xf3(\x7f\x9b&\xb0!Ne\xb4\xac\xe5\'\xc4P\xa5\x13OU\xeeK\x03O\xcez\x91,xB\t\xab\x1e\xab\xc2\xed\x05\x8a\xe3T\xcd6\xd5\x9ff\xcc\xa5\x94\xedT\x06\xf2!\xea\xe2\xe3\x03\xcc\xf5\xe9\xfb \x97iD\x03\x82\xd1\xda\n\x06\x80\x03K\x18\xef\xe1\x05\\\x00\x82Zy\x89\xeb\x9c\x05\xd7{\x96\xa3x\xed\x92\xc4\xbb\xe4\xec\xf2W\xa4\xd8u49\x90\x866VH\xb1\xe1\x8b\xad\x0f\xa7+\xb8\xd8\x048\xc6\xdf\x804\xc0\xdd\xbf\x1da\xbd\x03\xc8/\x88\xc4m\xb0:\xcf8\xcb\xea\x91&o\xc1\xd7\xf3\r\xfb(\x8bZ'
Compressed size: 250 bytes
Decompressed string: CBHHBd%AbC%GJIEd%AbC%AAEEBd%AbC%IGJDBd%AbC%EAHGBd%AbC%EEFEBd%AbC%GJIEd%AbC%CFDId%AbC%IAGEd%AbC%EABDBd%AbC%GJIEd%AbC%AACHd%AbC%CBJGd%AbC%AACHd%AbC%IIEHd%AbC%GJIEd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%GFAHd%AbC%CBJGd%AbC%GJIEd%Ab

In [11]:
import bz2

# Original string
original_string = key

# Compress the string using bz2
compressed_data = bz2.compress(original_string.encode())

print(f"Compressed data (bz2): {compressed_data}")
print(f"Compressed size (bz2): {len(compressed_data)} bytes")

# Decompress the string
decompressed_data = bz2.decompress(compressed_data).decode()

print(f"Decompressed string (bz2): {decompressed_data}")
print(f"Is decompressed string the same as original? {decompressed_data == original_string}")


Compressed data (bz2): b'BZh91AY&SY\x1d\x0f\x1c3\x00\x01_\x15$\x02\x00?\xf0\x14\x00@\x00\x10\x000\x01z\xb5m\xb0Ja*6\x91\xa0I\xa8\x82d\x80*D\xa7\xa3\xd4L\x8d\x11\xe0B\x89$\xb2k\x85\xcb\x93\xb8\xb9e\\MF\x02\xde\x02\x00\xc0g\xa32\x17X\xce\x08P\x91\xa2\x0b\xb6\xa6\x17\xf1\xf3Y\xe0C\'\xbd\x86\xeaJ\x9a%\xca\x9e\xce\x9e\xad\xd8\xf2\x18\x01\xf6\x1b\x93\xdeK\x83a\xc0aPC\x19a\x84\xb16\xac\x8aT*\x15C\x17\xd3L\xba\xe62-k6\xf20r\x11\x03\x86\xa5C\xb0\xd4\xa2\x13y\x0e\xaa)\x90\x1b\xf3\xc8wP\x98m\xc8\xf7\x95\xb7\xa3Q\x93M[\xb2\xd9\xbb_F\x03\xe9s\x08`\x9c\xcb\x14\xe1\xba\xaaq\x16\xc9\xa7+t\x8d\x16\'[\x04\r\x94[U\x99L\xb8\xd5)j\xc5\x16\xec"\xa1\xacb\xe0\x88\x99[\x18\xd2(\xca\xa4IX\xb1,\x11DB\xc8TX\x8a\xc1bXP\xe4\x9a\xf7\x7fw\xfc]\xc9\x14\xe1B@t<p\xcc'
Compressed size (bz2): 257 bytes
Decompressed string (bz2): CBHHBd%AbC%GJIEd%AbC%AAEEBd%AbC%IGJDBd%AbC%EAHGBd%AbC%EEFEBd%AbC%GJIEd%AbC%CFDId%AbC%IAGEd%AbC%EABDBd%AbC%GJIEd%AbC%AACHd%AbC%CBJGd%AbC%AACHd%AbC%IIEHd%AbC%GJIEd%AbC%GDDGd%AbC%IAGEd%AbC%GJIEd%AbC%

In [12]:
import lzma

# Original string
original_string = key

# Compress the string using lzma
compressed_data = lzma.compress(original_string.encode())

print(f"Compressed data (lzma): {compressed_data}")
print(f"Compressed size (lzma): {len(compressed_data)} bytes")

# Decompress the string
decompressed_data = lzma.decompress(compressed_data).decode()

print(f"Decompressed string (lzma): {decompressed_data}")
print(f"Is decompressed string the same as original? {decompressed_data == original_string}")


Compressed data (lzma): b'\xfd7zXZ\x00\x00\x04\xe6\xd6\xb4F\x02\x00!\x01\x16\x00\x00\x00t/\xe5\xa3\xe0\x05\xc6\x01\x02]\x00!\x90\x85C(\x0eA\xaeoj\x8c\xb071N\xd5\xc1\x8d\xe8J\xb5\xea\x97\xcc\xce\xbce\xc0\x80\xa7\x90M\xcc\xb2-\xc2\x98Q\xf1\xc4A\x8e\xd7\xef\x15Q_ `\x1b\x7f\x1c\xbf\x91o|-\xd3\x8f\xb2\xf1x\x8f\x0c\x1d\xaco\xd9F\xc2r\xf2\xc0\xdcwA=pB\xb3\xd6\x9a\xcc\xf7\x1c\xe50Td\x8a\xdey)l\xe4{s\xbc\xaf\xbf\xbf(\xd3k\x02^\xbb\x07\x8bv\x828Kj\xa7\r?\x80\xbb\x1d4*\xd8?\xe7\x10z\xe0\x15\x15pC\x9b}3h+s\x92\x03\xd9\x84N\x02\x98Mzm\xef\x19-\xfe\xf4\xac08\x90Z\xf8\x99i\xc2yY\x97,bz\x7f\xb7\x8f@\xba\x8c\xc9`\x0bc\xb7\x93(\xbe]l+\x8cUb\xabX~\xa4\xd4\xd4\x9d\xc4Wj4\x19\xae\xd7\xde\x11\x9b\xb0\x9f\x05\x84\x9e\xe9N*\x07\xecK\r\x8d&\xb8\xa1u\x88\x7f:4\xc7zkG\x05\xc6\x85\x87\xe6\x143\xd1\x85\x03S7\x87>f+t\xf4\x0e\x1b+/\x19\xff\x81H\x00\x00\x00\x00\x00\xd9A\xe62\xf1\x9f\x9d\xff\x00\x01\x9e\x02\xc7\x0b\x00\x00\xd0\xc96y\xb1\xc4g\xfb\x02\x00\x00\x00\x00\x04YZ'
Compressed size (lzma): 324 bytes
Decompressed